In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from math import log, sqrt, exp
from scipy.optimize import brentq, fsolve


import warnings
warnings.filterwarnings("ignore")

In [2]:
# import the dataset
swaption_df = pd.read_excel('IR Data.xlsx', sheet_name = 'Swaption', skiprows = 2)
swaption_df.index = swaption_df[['Expiry', 'Tenor']]
swaption_df.drop(['Expiry', 'Tenor'], inplace=True, axis = 1)
swaption_df = swaption_df / 100
swaption_df

,-200bps,-150bps,-100bps,-50bps,-25bps,ATM,+25bps,+50bps,+100bps,+150bps,+200bps
"(1Y, 1Y)",0.91570,0.62030,0.44130,0.31224,0.26182,0.2250,0.2096,0.2140,0.2434,0.27488,0.30297
"(1Y, 2Y)",0.83270,0.61240,0.46570,0.35807,0.31712,0.2872,0.2712,0.2684,0.2851,0.31025,0.33523
"(1Y, 3Y)",0.73920,0.56870,0.44770,0.35745,0.32317,0.2978,0.2829,0.2780,0.2877,0.30725,0.32833
"(1Y, 5Y)",0.55190,0.44640,0.36510,0.30242,0.27851,0.2607,0.2498,0.2456,0.2512,0.26536,0.28165
"(1Y, 10Y)",0.41180,0.35040,0.30207,0.26619,0.25351,0.2447,0.2398,0.2382,0.2425,0.25204,0.26355
"(5Y, 1Y)",0.67800,0.49090,0.38400,0.31485,0.29060,0.2726,0.2604,0.2532,0.2494,0.25320,0.25980
"(5Y, 2Y)",0.57880,0.46410,0.39033,0.33653,0.31531,0.2983,0.2856,0.2765,0.2671,0.26540,0.26760
"(5Y, 3Y)",0.53430,0.44440,0.38180,0.33437,0.31536,0.2998,0.2876,0.2782,0.2667,0.26200,0.26150
"(5Y, 5Y)",0.41990,0.36524,0.32326,0.29005,0.27677,0.2660,0.2573,0.2502,0.2406,0.23570,0.23400
"(5Y, 10Y)",0.34417,0.30948,0.28148,0.25954,0.25136,0.2451,0.2399,0.2356,0.2291,0.22490,0.22250


In [3]:
# import forward swap rate from part 1
swap_rate_df = pd.read_csv('forward_swap_rate.csv')
swap_rate_df.set_index('tenor', drop = True, inplace=True)
discount_factor_df = pd.read_csv('Discount_Factors.csv')
discount_factor_df.set_index('Tenor', drop=True, inplace=True)

In [4]:
swap_rate_df

,1Y,5Y,10Y
tenor,,,
1Y,0.032007,0.039274,0.042189
2Y,0.033259,0.040075,0.043116
3Y,0.034011,0.040072,0.044097
5Y,0.035255,0.041093,0.046249
10Y,0.038428,0.043634,0.053458


In [5]:
bps = [-0.02, -0.015, -0.01, -0.005, -0.0025, 0, 0.0025, 0.005, 0.01, 0.015, 0.02]

## Displaced-Diffusion Model Calibration 

In [6]:
# Black76 Model
def Black76Call(F, K, sigma, T):
    d1 = (np.log(F / K) + (1 / 2) * (sigma ** 2) * T) / (sigma * sqrt(T))
    d2 = d1 - sigma * sqrt(T)
    V = (F * norm.cdf(d1) - K * norm.cdf(d2))
    return V

def Black76Put(F, K, sigma, T):
    d1 = (np.log(F / K) + (1 / 2) * (sigma ** 2) * T) / (sigma * sqrt(T))
    d2 = d1 - sigma * sqrt(T)
    V = (K * norm.cdf(-d2) - F * norm.cdf(-d1))
    return V

def Black76(F, K, sigma, T):
    if F <= K: 
        d1 = np.log(F / K) + (0.5 * (sigma ** 2) * T) / (sigma * sqrt(T))
        d2 = d1 - sigma * sqrt(T)
        V = F * norm.cdf(d1) - K * norm.cdf(d2)
    elif F > K: 
        d1 = (np.log(F / K) + (0.5 * (sigma ** 2) * T) / (sigma * sqrt(T)))
        d2 = d1 - sigma * sqrt(T)
        V = K * norm.cdf(-d2) - F* norm.cdf(-d1)
    else:
        raise NameError('Payoff Type not Recognized')
    return V

# Displaced Diffusion Model

def DisplaceDiffusion(F, K, pvbp, sigma, T, beta):
    if F <= K:
        F_new = F / beta
        K_new = K + ((1 - beta)/beta) * F_new
        sigma_new = sigma * beta
        V = pvbp * Black76(F_new, K_new, sigma_new, T)
    elif F > K:
        F_new = F / beta
        K_new = K + ((1 - beta) / beta) * F_new
        sigma_new = sigma * beta
        V = pvbp * Black76(F_new, K_new, sigma_new, T)
    else:
        raise NameError('Payoff Type not Recognized')
    return V

def impliedVolatility(F, K, pvbp, sigma, T, beta):
    
    price = DisplaceDiffusion(F, K, pvbp, sigma, T, beta)
    impliedVol = fsolve(lambda x: price -
                         pvbp*Black76(F, K, x, T),
                        0.5)
    return impliedVol[0]    


def ddcalibration(x, F, strikes, pvbp, vols, T):  # x: [sigma,beta]
    err = 0.0
    for i, vol in enumerate(vols):
        err += (vol - impliedVolatility(F, strikes[i], pvbp,x[0], T, x[1] ))**2
    return err

# def ddcalibration(x, F, strikes, pvbp, vols, T):
#     err = 0.0
#     sigma = vols[5]
#     for i, vol in enumerate(vols):
#         err += (vol - impliedVolatility(F, strikes[i], pvbp,sigma, T, x))**2
#     return err

In [7]:
expiry_lst = ['1Y', '5Y', '10Y']
tenor_lst = ['1Y', '2Y', '3Y', '5Y', '10Y']

In [8]:
swap_rate_df.T

tenor,1Y,2Y,3Y,5Y,10Y
1Y,0.032007,0.033259,0.034011,0.035255,0.038428
5Y,0.039274,0.040075,0.040072,0.041093,0.043634
10Y,0.042189,0.043116,0.044097,0.046249,0.053458


In [9]:
swaption_df

,-200bps,-150bps,-100bps,-50bps,-25bps,ATM,+25bps,+50bps,+100bps,+150bps,+200bps
"(1Y, 1Y)",0.91570,0.62030,0.44130,0.31224,0.26182,0.2250,0.2096,0.2140,0.2434,0.27488,0.30297
"(1Y, 2Y)",0.83270,0.61240,0.46570,0.35807,0.31712,0.2872,0.2712,0.2684,0.2851,0.31025,0.33523
"(1Y, 3Y)",0.73920,0.56870,0.44770,0.35745,0.32317,0.2978,0.2829,0.2780,0.2877,0.30725,0.32833
"(1Y, 5Y)",0.55190,0.44640,0.36510,0.30242,0.27851,0.2607,0.2498,0.2456,0.2512,0.26536,0.28165
"(1Y, 10Y)",0.41180,0.35040,0.30207,0.26619,0.25351,0.2447,0.2398,0.2382,0.2425,0.25204,0.26355
"(5Y, 1Y)",0.67800,0.49090,0.38400,0.31485,0.29060,0.2726,0.2604,0.2532,0.2494,0.25320,0.25980
"(5Y, 2Y)",0.57880,0.46410,0.39033,0.33653,0.31531,0.2983,0.2856,0.2765,0.2671,0.26540,0.26760
"(5Y, 3Y)",0.53430,0.44440,0.38180,0.33437,0.31536,0.2998,0.2876,0.2782,0.2667,0.26200,0.26150
"(5Y, 5Y)",0.41990,0.36524,0.32326,0.29005,0.27677,0.2660,0.2573,0.2502,0.2406,0.23570,0.23400
"(5Y, 10Y)",0.34417,0.30948,0.28148,0.25954,0.25136,0.2451,0.2399,0.2356,0.2291,0.22490,0.22250


In [10]:
from scipy.optimize import least_squares

initialGuess = [0.2,0.2]
k = 0
params_lst = []
# beta_lst = []
# vol_lst = []

freq = 0.5

for i in range(len(expiry_lst)):
    for j in range(len(tenor_lst)):
        forward = swap_rate_df.T.iloc[i, j]
        vol = swaption_df.iloc[k, :].values
        k = k + 1
        strike = forward + bps
        expiry_T = int(expiry_lst[i].replace('Y', ''))
        tenor_T = int(tenor_lst[j].replace('Y', ''))
        pvbp = freq * discount_factor_df.iloc[2 * expiry_T:2*(expiry_T + tenor_T), 0].values.sum()
        res = least_squares(lambda x: ddcalibration(x,
                                                    forward, 
                                                    strike,
                                                    pvbp,
                                                    vol,
                                                    expiry_T), 
                            initialGuess,
                             bounds = ((0, 0), (1, 1)),
#                            bounds=[(0,0),(np.inf, 1)], 
#                            method = 'trf'
                           )
        params_lst.append(res.x)
        # beta_lst.append(res.x[0])
        # vol_lst.append(vol[5])
        
        

In [11]:
params_lst

[array([0.40886412, 0.66658703]),
 array([0.39485651, 0.99999998]),
 array([0.41028683, 0.737634  ]),
 array([0.34307574, 0.76569367]),
 array([4.00098820e-01, 9.88693409e-05]),
 array([0.32432843, 0.68098757]),
 array([0.32659888, 0.70684838]),
 array([0.31982461, 0.71706731]),
 array([0.2, 0.2]),
 array([0.69006978, 0.39563201]),
 array([0.5850752 , 0.39938832]),
 array([0.30262158, 0.71232105]),
 array([0.29936033, 0.72192202]),
 array([0.58574116, 0.39043458]),
 array([0.55685092, 0.40111462])]

In [12]:
alpha_lst = [param[0] for param in params_lst]
beta_lst = [param[1] for param in params_lst]

In [13]:
y1_alpha_lst =  alpha_lst[:5]
y5_alpha_lst =  alpha_lst[5:10]
y10_alpha_lst =  alpha_lst[10:]

y1_beta_lst =  beta_lst[:5]
y5_beta_lst =  beta_lst[5:10]
y10_beta_lst =  beta_lst[10:]

In [14]:
Beta_DD = pd.DataFrame()
Beta_DD['1Y'] = y1_beta_lst 
Beta_DD['5Y'] = y5_beta_lst 
Beta_DD['10Y'] = y10_beta_lst 
Beta_DD['Tenor'] = tenor_lst
Beta_DD.set_index('Tenor', drop=True, inplace=True)

print(Beta_DD)

             1Y        5Y       10Y
Tenor                              
1Y     0.666587  0.680988  0.399388
2Y     1.000000  0.706848  0.712321
3Y     0.737634  0.717067  0.721922
5Y     0.765694  0.200000  0.390435
10Y    0.000099  0.395632  0.401115


In [15]:
Beta_DD

,1Y,5Y,10Y
Tenor,,,
1Y,0.666587,0.680988,0.399388
2Y,1.000000,0.706848,0.712321
3Y,0.737634,0.717067,0.721922
5Y,0.765694,0.200000,0.390435
10Y,0.000099,0.395632,0.401115


In [16]:
y1_alpha_lst = alpha_lst[:5]
y5_alpha_lst = alpha_lst[5:10]
y10_alpha_lst = alpha_lst[10:]

Alpha_DD = pd.DataFrame()
Alpha_DD['Tenor'] = tenor_lst
Alpha_DD['1Y'] = y1_alpha_lst
Alpha_DD['5Y'] = y5_alpha_lst
Alpha_DD['10Y'] = y10_alpha_lst
Alpha_DD.set_index('Tenor', drop=True, inplace=True)

print(Alpha_DD)

             1Y        5Y       10Y
Tenor                              
1Y     0.408864  0.324328  0.585075
2Y     0.394857  0.326599  0.302622
3Y     0.410287  0.319825  0.299360
5Y     0.343076  0.200000  0.585741
10Y    0.400099  0.690070  0.556851


## Question 2: SABR Model Calibration (Fixed $\beta$ = 0.9) 

In [17]:
# SABR fixed beta = 0.9
def SABR(F, K, T, alpha, beta, rho, nu):
    X = K
    # if K is at-the-money-forward
    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta)**2)/24)*alpha*alpha/(F**(2 - 2*beta))
        numer2 = 0.25*rho*beta*nu*alpha/(F**(1 - beta))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        VolAtm = alpha*(1 + (numer1 + numer2 + numer3)*T)/(F**(1-beta))
        sabrsigma = VolAtm
    else:
        z = (nu/alpha)*((F*X)**(0.5*(1-beta)))*np.log(F/X)
        zhi = np.log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
        numer1 = (((1 - beta)**2)/24)*((alpha*alpha)/((F*X)**(1 - beta)))
        numer2 = 0.25*rho*beta*nu*alpha/((F*X)**((1 - beta)/2))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        numer = alpha*(1 + (numer1 + numer2 + numer3)*T)*z
        denom1 = ((1 - beta)**2/24)*(np.log(F/X))**2
        denom2 = (((1 - beta)**4)/1920)*((np.log(F/X))**4)
        denom = ((F*X)**((1 - beta)/2))*(1 + denom1 + denom2)*zhi
        sabrsigma = numer/denom

    return sabrsigma

In [18]:
beta = 0.9

def sabrcalibration(x, strikes, vols, F, T):
    err = 0.0
    for i, vol in enumerate(vols):
        err += (vol - SABR(F, strikes[i], T,
                           x[0], beta, x[1], x[2]))**2

    return err

In [19]:
expiry_lst = ['1Y', '5Y', '10Y']
tenor_lst = ['1Y', '2Y', '3Y', '5Y', '10Y']

In [20]:
from scipy.optimize import least_squares

initialGuess = [0.2, -0.5, 0.5]
bps = [-0.02, -0.015, -0.01, -0.005, -0.0025, 0, 0.0025, 0.005, 0.01, 0.015, 0.02]
k = 0
alpha_lst = []
rho_lst = []
nu_lst = []
for i in range(len(expiry_lst)):
    for j in range(len(tenor_lst)):
        forward = swap_rate_df.T.iloc[i, j]
        vol = swaption_df.iloc[k, :].values
        k = k + 1
        strike = forward + bps
        T = int(expiry_lst[i].replace('Y', ''))
        res = least_squares(lambda x: sabrcalibration(x,
                                              strike,
                                              vol,
                                              forward,
                                              T),
                            initialGuess)
        alpha_lst.append(res.x[0])
        rho_lst.append(res.x[1])
        nu_lst.append(res.x[2])
        

In [21]:
y1_alpha_lst = alpha_lst[:5]
y5_alpha_lst = alpha_lst[5:10]
y10_alpha_lst = alpha_lst[10:]

y1_rho_lst = rho_lst[:5]
y5_rho_lst = rho_lst[5:10]
y10_rho_lst = rho_lst[10:]

y1_nu_lst = nu_lst[:5]
y5_nu_lst = nu_lst[5:10]
y10_nu_lst = nu_lst[10:]

In [22]:
Alpha_SABR = pd.DataFrame()
Alpha_SABR['1Y'] = y1_alpha_lst
Alpha_SABR['5Y'] = y5_alpha_lst
Alpha_SABR['10Y'] = y10_alpha_lst
Alpha_SABR['Tenor'] = tenor_lst
Alpha_SABR.set_index('Tenor', drop=True, inplace=True)
Rho_SABR = pd.DataFrame()
Rho_SABR['1Y'] = y1_rho_lst
Rho_SABR['5Y'] = y5_rho_lst
Rho_SABR['10Y'] = y10_rho_lst
Rho_SABR['Tenor'] = tenor_lst
Rho_SABR.set_index('Tenor', drop=True, inplace=True)
Nu_SABR = pd.DataFrame()
Nu_SABR['1Y'] = y1_nu_lst
Nu_SABR['5Y'] = y5_nu_lst
Nu_SABR['10Y'] = y10_nu_lst
Nu_SABR['Tenor'] = tenor_lst
Nu_SABR.set_index('Tenor', drop=True, inplace=True)

In [23]:
Alpha_SABR = Alpha_SABR.T
Rho_SABR = Rho_SABR.T
Nu_SABR = Nu_SABR.T

In [24]:
Alpha_SABR

Tenor,1Y,2Y,3Y,5Y,10Y
1Y,0.145508,0.189933,0.204179,0.187133,0.180561
5Y,0.163950,0.199100,0.211737,0.195295,0.179563
10Y,0.173473,0.190608,0.201316,0.196593,0.181919


In [25]:
Rho_SABR

Tenor,1Y,2Y,3Y,5Y,10Y
1Y,-0.608816,-0.525193,-0.496632,-0.461434,-0.377000
5Y,-0.564775,-0.540816,-0.550889,-0.533017,-0.466557
10Y,-0.528502,-0.528894,-0.533273,-0.545831,-0.512094


In [26]:
Nu_SABR

Tenor,1Y,2Y,3Y,5Y,10Y
1Y,1.933390,1.623749,1.376043,0.979113,0.646509
5Y,1.305390,1.051516,0.927147,0.652343,0.482230
10Y,0.987294,0.909526,0.852661,0.710866,0.569442


In [27]:
Alpha_SABR.to_csv('Alpha_SABR.csv')
Rho_SABR.to_csv('Rho_SABR.csv')
Nu_SABR.to_csv('Nu_SABR.csv')

## Question 3: Price the Swaptions Using Calibrated Displaced-Diffusion and SABR Models

In [28]:
def forward_swap_rate(expiry, tenor):
    freq = 0.5
    fixed_leg = freq * (discount_factor_df.iloc[2*expiry:2*(expiry+tenor), 0].sum())
    float_leg = 0
    for k in range(2 * expiry, 2 * (expiry + tenor)):
        float_leg = float_leg + (discount_factor_df.iloc[k, 0] * \
                                ((discount_factor_df.iloc[k-1, 1] - discount_factor_df.iloc[k, 1]) / discount_factor_df.iloc[k, 1]))
    forward_swap_rate = float_leg / fixed_leg
    return forward_swap_rate

### 1）Calibrated Displaced-Diffusion Model

#### payer 2y × 10y K = 1\%,2\%,3\%,4\%,5\%,6\%,7\%,8\%

In [29]:
def interpol(x1,y1,x2,y2,x):
    return y1+(x-x1)*((y2-y1)/(x2-x1))

In [ ]:
DisplaceDiffusion(F, K, pvbp, sigma, T, beta)

In [43]:
def price_swaption_DD(payoff_type, expiry, tenor, strike_rates, alpha, beta):
    
    forward_rate = forward_swap_rate(expiry, tenor)
   
    pvbp = 0.5 * discount_factor_df.iloc[2*expiry:2*(expiry+tenor), 0].values.sum()
    prices = []
    for K in strike_rates:
        if payoff_type == "payer":
            price = DisplaceDiffusion(forward_rate, K, pvbp, alpha, expiry, beta)
        elif payoff_type == "receiver":
            price = DisplaceDiffusion(forward_rate, K, pvbp, alpha, expiry, beta)
        prices.append(price)
    return prices

In [44]:
beta_payer_2y10y = Beta_DD.loc['2Y', '10Y']
alpha_payer_2y10y = Alpha_DD.loc['2Y', '10Y']

In [45]:
strike_rates = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08]

prices_payer_2y10y = price_swaption_DD("payer", 2, 10, strike_rates, alpha_payer_2y10y, beta_payer_2y10y)

In [46]:
Payer_DD_df = pd.DataFrame()
Payer_DD_df['Strikes'] = strike_rates
Payer_DD_df['DD_Price'] = prices_payer_2y10y
Payer_DD_df

,Strikes,DD_Price
0,0.01,-0.021970
1,0.02,0.005171
2,0.03,0.048751
3,0.04,0.038830
4,0.05,0.007907
5,0.06,-0.015585
6,0.07,-0.033389
7,0.08,-0.046820


####  Receiver 8y × 10y K = 1\%,2\%,3\%,4\%,5\%,6\%,7\%, 8\%

In [34]:
x_points = [5, 10]  # 已知点的年份
y_beta_points = [Beta_DD.loc['5Y', '10Y'], Beta_DD.loc['10Y', '10Y']]  # 已知点的beta值，假设5Y和10Y的beta值
y_alpha_points = [Alpha_DD.loc['5Y', '10Y'], Alpha_DD.loc['10Y', '10Y']]  # 已知点的alpha值，假设5Y和10Y的alpha值

x_target = 8

beta_receiver_8y10y = np.interp(x_target, x_points, y_beta_points)
alpha_receiver_8y10y = np.interp(x_target, x_points, y_alpha_points)

print(beta_receiver_8y10y)
print(alpha_receiver_8y10y)

0.39684260041220537
0.5684070179116039


In [35]:
prices_receiver_8y10y = price_swaption_DD("receiver", 8, 10, strike_rates, alpha_receiver_8y10y, beta_receiver_8y10y)

In [36]:
Receiver_DD_df = pd.DataFrame()
Receiver_DD_df['Strikes'] = strike_rates
Receiver_DD_df['DD_Price'] = prices_receiver_8y10y
Receiver_DD_df

,Strikes,DD_Price
0,0.01,0.035924
1,0.02,0.039092
2,0.03,0.028762
3,0.04,0.021410
4,0.05,0.016286
5,0.06,0.012649
6,0.07,0.010005
7,0.08,0.008040


### 2) Calibrated SABR Models 

#### payer 2y × 10y K = 1\%,2\%,3\%,4\%,5\%,6\%,7\%,8\%

In [37]:
# Interpolate alpha, rho and nu
alpha5 = Alpha_SABR.loc['5Y', '10Y']
alpha10 = Alpha_SABR.loc['10Y', '10Y']
alpha_810 = ((alpha10 - alpha5) / (10-5) * (8-5)) + alpha5

rho5 = Rho_SABR.loc['5Y', '10Y']
rho10 = Rho_SABR.loc['10Y', '10Y']
rho_810 = ((rho10 - rho5) / (10-5) * (8-5)) + rho5

nu5 = Nu_SABR.loc['5Y', '10Y']
nu10 = Nu_SABR.loc['10Y', '10Y']
nu_810 = ((nu10 - nu5) / (10-5) * (8-5)) + nu5

In [38]:
beta = 0.9
Frate_210 = forward_swap_rate(2, 10)
K = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08]
T = 2
pvbp1 = 0.5 * discount_factor_df.iloc[2*2:2*(10+2), 0].values.sum()
SABR_lst = []
for i in K: 
    price = pvbp1 * Black76Call(Frate_210, i, SABR(Frate_210, i, T, alpha_810, beta, rho_810, nu_810), T)
    SABR_lst.append(price)

In [39]:
Payer_SABR_df = pd.DataFrame()
Payer_SABR_df['Strikes'] = K
Payer_SABR_df['SABR_Price'] = SABR_lst
Payer_SABR_df

,Strikes,SABR_Price
0,0.01,0.288623
1,0.02,0.196740
2,0.03,0.114454
3,0.04,0.052169
4,0.05,0.018597
5,0.06,0.006441
6,0.07,0.002616
7,0.08,0.001267


####  Receiver 8y × 10y K = 1\%,2\%,3\%,4\%,5\%,6\%,7\%, 8\%

In [40]:
# Interpolate alpha, rho and nu
alpha5 = Alpha_SABR.loc['5Y', '10Y']
alpha10 = Alpha_SABR.loc['10Y', '10Y']
alpha_810 = ((alpha10 - alpha5) / (10-5) * (8-5)) + alpha5

rho5 = Rho_SABR.loc['5Y', '10Y']
rho10 = Rho_SABR.loc['10Y', '10Y']
rho_810 = ((rho10 - rho5) / (10-5) * (8-5)) + rho5

nu5 = Nu_SABR.loc['5Y', '10Y']
nu10 = Nu_SABR.loc['10Y', '10Y']
nu_810 = ((nu10 - nu5) / (10-5) * (8-5)) + nu5

In [41]:
beta = 0.9
Frate_810 = forward_swap_rate(8, 10)
K = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08]
T = 8
pvbp2 = 0.5 * discount_factor_df.iloc[2*8:2*(10+2), 0].values.sum()
SABR_lst = []
for i in K: 
    price = pvbp1 * Black76Put(Frate_810, i, SABR(Frate_810, i, T, alpha_810, beta, rho_810, nu_810), T)
    SABR_lst.append(price)

In [42]:
Receiver_SABR_df = pd.DataFrame()
Receiver_SABR_df['Strikes'] = K
Receiver_SABR_df['SABR_Price'] = SABR_lst
Receiver_SABR_df

,Strikes,SABR_Price
0,0.01,0.019265
1,0.02,0.039012
2,0.03,0.062656
3,0.04,0.093088
4,0.05,0.134714
5,0.06,0.192162
6,0.07,0.265302
7,0.08,0.348860
